# Load packages, read-in pre-processed data

In [1]:
#Import relevant packages

import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

from bokeh.io import show, curdoc, output_notebook
from bokeh.layouts import column
from bokeh.models import (
    ColumnDataSource,
    Label,
    LabelSet,
    CheckboxGroup,
    CustomJS,
    Button,
)
from bokeh.models.annotations import LabelSet
from bokeh.palettes import Category10
from bokeh.plotting import figure

import numpy as np

In [2]:
#  Read-in the pre-processed data

# df without urban-rural classification
df = pd.read_csv('/Users/loucap/Documents/GitWork/InteractiveGender/Data/cleaned_lang_SO.csv')

# Let's take a quick glance

df.head()

,LA_code,LA_name,SO_code,SO_categories,Observation_x,Percentages,Non-response_rate,region_x,Observation_y,Observation_total,Percentage,region_y,Shannon_idx
0,E06000001,Hartlepool,5,Not answered,4554,6.097528,6.097528,North East,1875,92337,2.030605,North East,0.931876
1,E06000002,Middlesbrough,5,Not answered,8298,7.282908,7.282908,North East,10510,143923,7.302516,North East,1.181237
2,E06000003,Redcar and Cleveland,5,Not answered,7046,6.271920,6.271920,North East,1460,136533,1.069339,North East,0.831589
3,E06000004,Stockton-on-Tees,5,Not answered,9268,5.865452,5.865452,North East,5674,196603,2.886019,North East,1.001650
4,E06000005,Darlington,5,Not answered,5010,5.686332,5.686332,North East,4403,107800,4.084416,North East,0.966864


In [3]:
# df with urban-rural classification
df2 = pd.read_csv('/Users/loucap/Documents/GitWork/InteractiveGender/Data/urban_rural_SO.csv')

# Let's take a quick glance
# IMPORTANT: we only have urb_rural classification for ENGLISH LA's
df2.head()

,LA_code,LA_name,SO_code,SO_categories,Observation_x,Percentages,Non-response_rate,region_x,Observation_y,Observation_total,Percentage,region_y,Urb_Rur
0,E06000001,Hartlepool,5,Not answered,4554,6.097528,6.097528,North East,1875,92337,2.030605,North East,Predominantly Urban
1,E06000002,Middlesbrough,5,Not answered,8298,7.282908,7.282908,North East,10510,143923,7.302516,North East,Predominantly Urban
2,E06000003,Redcar and Cleveland,5,Not answered,7046,6.271920,6.271920,North East,1460,136533,1.069339,North East,Urban with Significant Rural
3,E06000004,Stockton-on-Tees,5,Not answered,9268,5.865452,5.865452,North East,5674,196603,2.886019,North East,Predominantly Urban
4,E06000005,Darlington,5,Not answered,5010,5.686332,5.686332,North East,4403,107800,4.084416,North East,Predominantly Urban


# Interactive scatterplots

## Shows the relationship between the % of Non-English speakers and % of Non-response for our 331 Local Authorities in England and Wales.

### COLOURED BY REGION

In [4]:
df['Urb_Rur'] = df2['Urb_Rur']

In [5]:
df

,LA_code,LA_name,SO_code,SO_categories,Observation_x,Percentages,Non-response_rate,region_x,Observation_y,Observation_total,Percentage,region_y,Shannon_idx,Urb_Rur
0,E06000001,Hartlepool,5,Not answered,4554,6.097528,6.097528,North East,1875,92337,2.030605,North East,0.931876,Predominantly Urban
1,E06000002,Middlesbrough,5,Not answered,8298,7.282908,7.282908,North East,10510,143923,7.302516,North East,1.181237,Predominantly Urban
2,E06000003,Redcar and Cleveland,5,Not answered,7046,6.271920,6.271920,North East,1460,136533,1.069339,North East,0.831589,Urban with Significant Rural
3,E06000004,Stockton-on-Tees,5,Not answered,9268,5.865452,5.865452,North East,5674,196603,2.886019,North East,1.001650,Predominantly Urban
4,E06000005,Darlington,5,Not answered,5010,5.686332,5.686332,North East,4403,107800,4.084416,North East,0.966864,Predominantly Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,W06000020,Torfaen,5,Not answered,5233,6.966837,6.966837,Wales,1318,92273,1.428370,Wales,0.850490,Welsh/not specified
327,W06000021,Monmouthshire,5,Not answered,5423,6.931237,6.931237,Wales,1457,92955,1.567425,Wales,0.855804,Welsh/not specified
328,W06000022,Newport,5,Not answered,8721,6.844133,6.844133,Wales,10035,159590,6.287988,Wales,1.085068,Welsh/not specified
329,W06000023,Powys,5,Not answered,9832,8.736837,8.736837,Wales,2496,133173,1.874254,Wales,0.861874,Welsh/not specified


In [6]:
df.Urb_Rur.unique()

array(['Predominantly Urban', 'Urban with Significant Rural',
       'Predominantly Rural', 'Not specified', 'Welsh/not specified'],
      dtype=object)

In [7]:
# !pip install --upgrade bokeh


In [27]:
import pandas as pd
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure, curdoc
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap
from bokeh.models import ColorBar, BasicTicker, PrintfTickFormatter, LogColorMapper

# Prepare data sources
df['Urb_Rur'] = df['Urb_Rur'].astype(str)
source = ColumnDataSource(df)


# Define tooltips
tool = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("name", "@LA_name"),
]

p0 = figure(title = "Relationship between Non-response Rate and Non-English Speakers", x_axis_label = "Percentage of Non-English Speakers",
           y_axis_label = "Non-response rate", tooltips = tool)

p0.scatter("Percentage", "Non-response_rate", source=source, fill_alpha=0.5, size=10)

# Plot 1 (By Region)
p1 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Percentage of Non-English Speakers",
            y_axis_label="Non-response Rate",
            tooltips=tool)

for region, color in zip(df.region_x.unique(), Category10[10]):
    b = df[df.region_x == region]
    p1.circle(x='Percentage', y='Non-response_rate', size=10, alpha=0.5, color=color,
              legend_label=region, muted_color=color, muted_alpha=0.1, source=ColumnDataSource(b))

p1.legend.location = "bottom_right"
p1.legend.click_policy = "hide"
p1.legend.title = "Regions"

# Plot 2 (Urban vs Rural)
p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Percentage of Non-English Speakers",
            y_axis_label="Non-response Rate",
            tooltips=tool)

urban_rural_sources = {}  # Create a dictionary to store the ColumnDataSource objects
for urb_rur in df.Urb_Rur.unique():
    urban_rural_sources[urb_rur] = ColumnDataSource(df[df.Urb_Rur == urb_rur])

for urb_rur, color in zip(df.Urb_Rur.unique(), Category10[10]):
    p2.circle(x='Percentage', y='Non-response_rate', size=10, alpha=0.5, color=color,
              legend_label=urb_rur, muted_color=color, muted_alpha=0.1, source=urban_rural_sources[urb_rur])

p2.legend.location = "bottom_right"
p2.legend.click_policy = "hide"
p2.legend.title = "Urban-Rural classification"



# Plot 3 (Shannon Index)
color_map = LogColorMapper(palette="Viridis256", low=df.Shannon_idx.min(), high=df.Shannon_idx.max())

p3 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Non-response Rate",
            y_axis_label="Percentage of Non-English Speakers",
            tooltips=tool)

p3.scatter("Percentage", "Non-response_rate", source=source, fill_alpha=0.5, size=10,
           color={'field': 'Shannon_idx', 'transform': color_map})

color_bar = ColorBar(color_mapper=color_map,
                     title='Shannon Index',
                     ticker=BasicTicker(desired_num_ticks=5),
                     formatter=PrintfTickFormatter(format='%.2f'))

p3.add_layout(color_bar, 'right')

# Dropdown menu
dropdown = Select(title="Color By:", value="None", options=["Default", "Region", "Urban", "Shannon Index"])

# Define the update function
def update_scatterplots(attr, old, new):
    if dropdown.value == "Default":
        p0.visible = True
        p1.visible = False
        p2.visible = False
        p3.visible = False
    elif dropdown.value == "Region":
        p0.visible = False
        p1.visible = True
        p2.visible = False
        p3.visible = False
    elif dropdown.value == "Urban":
        p0.visible = False
        p1.visible = False
        p2.visible = True
        p3.visible = False
    elif dropdown.value == "Shannon Index":
        p0.visible = False
        p1.visible = False
        p2.visible = False
        p3.visible = True

# Set initial visibility
p0.visible = True
p1.visible = False
p2.visible = False
p3.visible = False

# Add the callback to the dropdown menu
dropdown.on_change('value', update_scatterplots)

# Create a layout with the dropdown menu and the scatterplots
layout = column(dropdown, p0, p1, p2, p3)

# Add the layout to the document
curdoc().add_root(layout)

show(p2)

In [43]:
df.columns

Index(['LA_code', 'LA_name', 'SO_code', 'SO_categories', 'Observation_x',
       'Percentages', 'Non-response_rate', 'region_x', 'Observation_y',
       'Observation_total', 'Percentage', 'region_y', 'Shannon_idx',
       'Urb_Rur'],
      dtype='object')

In [60]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

# Assuming you already have the 'df' DataFrame with the necessary columns

# source = ColumnDataSource(df)

# Plot 2 (Urban vs Rural)
p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Percentage of Non-English Speakers",
            y_axis_label="Non-response Rate")

for urban, color in zip(df.Urb_Rur.unique(), Category10[10]):
    b = df[df.Urb_Rur == urban]
    p2.circle(x='Percentage', y='Non-response_rate', size=10, alpha=0.5, color=color,
              legend_label=urban, muted_color=color, muted_alpha=0.1, source=ColumnDataSource(b))


show(p2)


In [62]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

# Assuming you already have the 'df' DataFrame with the necessary columns

# source = ColumnDataSource(df)

# Plot 2 (Urban vs Rural)
p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Percentage of Non-English Speakers",
            y_axis_label="Non-response Rate")

for urban, color in zip(df.Urb_Rur.unique(), Category10[10]):
    b = df[df.Urb_Rur == urban]
    p2.circle(x='Percentage', y='Non-response_rate', size=10, alpha=0.5, color=color,
              legend_label=urban, muted_color=color, muted_alpha=0.1, source=ColumnDataSource(b))


show(p2)


In [64]:
from bokeh.plotting import figure, show
import numpy as np

x = np.random.rand(100)
y = np.random.rand(100)

p = figure(title="Scatter Plot Example")
p.scatter(x, y, size=10, color="red")

show(p)


In [51]:
b

,LA_code,LA_name,SO_code,SO_categories,Observation_x,Percentages,Non-response_rate,region_x,Observation_y,Observation_total,Percentage,region_y,Shannon_idx,Urb_Rur
309,W06000001,Isle of Anglesey,5,Not answered,4552,7.965144,7.965144,Wales,622,68876,0.903072,Wales,0.798022,Welsh/not specified
310,W06000002,Gwynedd,5,Not answered,9163,9.353054,9.353054,Wales,2855,117387,2.432126,Wales,0.874979,Welsh/not specified
311,W06000003,Conwy,5,Not answered,7986,8.287327,8.287327,Wales,2057,114750,1.792593,Wales,0.853679,Welsh/not specified
312,W06000004,Denbighshire,5,Not answered,6537,8.284331,8.284331,Wales,1610,95816,1.680304,Wales,0.847827,Welsh/not specified
313,W06000005,Flintshire,5,Not answered,9287,7.279468,7.279468,Wales,5518,154955,3.561034,Wales,0.807705,Welsh/not specified
314,W06000006,Wrexham,5,Not answered,8690,7.880371,7.880371,Wales,6682,135116,4.945380,Wales,0.863172,Welsh/not specified
315,W06000008,Ceredigion,5,Not answered,6405,10.433125,10.433125,Wales,2059,71471,2.880889,Wales,0.898527,Welsh/not specified
316,W06000009,Pembrokeshire,5,Not answered,8094,7.892658,7.892658,Wales,1636,123359,1.326210,Wales,0.843072,Welsh/not specified
317,W06000010,Carmarthenshire,5,Not answered,12307,7.915182,7.915182,Wales,4432,187897,2.358739,Wales,0.858149,Welsh/not specified
318,W06000011,Swansea,5,Not answered,15414,7.790475,7.790475,Wales,11023,238494,4.621919,Wales,1.012684,Welsh/not specified


In [49]:
source = ColumnDataSource(df)
# Plot 2 (Urban vs Rural)
p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
            x_axis_label="Percentage of Non-English Speakers",
            y_axis_label="Non-response Rate")


for urban, color in zip(df.Urb_Rur.unique(), Category10[10]):
    b = df[df.Urb_Rur == urban]
    p2.circle(x = 'Percentage', y = 'Non-response_rate', size = 10, alpha = 0.5, color = color, legend_label = str(df.Urb_Rur), muted_color = color, muted_alpha = 0.1, source = ColumnDataSource

show(p2)


RuntimeError: 

Expected x and y to reference fields in the supplied data source.

When a 'source' argument is passed to a glyph method, values that are sequences
(like lists or arrays) must come from references to data columns in the source.

For instance, as an example:

    source = ColumnDataSource(data=dict(x=a_list, y=an_array))

    p.circle(x='x', y='y', source=source, ...) # pass column names and a source

Alternatively, *all* data sequences may be provided as literals as long as a
source is *not* provided:

    p.circle(x=a_list, y=an_array, ...)  # pass actual sequences and no source



In [37]:
len(df.Urb_Rur)

331

In [9]:
for urb_rur, color in zip(df.Urb_Rur.unique(), Category10[10]):
    print(urb_rur, color)

Predominantly Urban #1f77b4
Urban with Significant Rural #ff7f0e
Predominantly Rural #2ca02c
Not specified #d62728
Welsh/not specified #9467bd


In [10]:
# import pandas as pd
# from bokeh.models import Toggle, ColumnDataSource, LabelSet
# from bokeh.plotting import figure, curdoc
# from bokeh.layouts import column
# from bokeh.palettes import Category10


# def create_layout():
    
#     source = ColumnDataSource(df)
#     tool = [
#         ("index", "$index"),
#         ("(x,y)", "($x, $y)"),
#         ("name", "@LA_name"),
#     ]

#     p1 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
#                 x_axis_label="Percentage of Non-English Speakers",
#                 y_axis_label="Non-response Rate",
#                 tooltips=tool)

#     circle_renderers = {}
#     labels_dict = {}

#     for region, color in zip(df.region_x.unique(), Category10[10]):
#         b = df[df.region_x == region]
#         r = p1.circle(x='Percentage', y='Non-response_rate', size=10, alpha=0.5, color=color,
#                       legend_label=region, muted_color=color, muted_alpha=0.1, source=ColumnDataSource(b))
#         circle_renderers[region] = r

#         labels = LabelSet(x='Percentage', y='Non-response_rate', text='LA_name',
#                           x_offset=5, y_offset=5, text_font_size="5pt",
#                           source=ColumnDataSource(b), text_alpha=0)
#         labels_dict[region] = labels
#         p1.add_layout(labels)

#     p1.legend.location = "bottom_right"
#     p1.legend.click_policy = "hide"
#     p1.legend.title = "Regions"

#     button = Toggle(label="Labels", button_type="primary", active=False)

#     def update_labels_visibility():
#         for legend_item in p1.legend.items:
#             renderer = legend_item.renderers[0]
#             region = legend_item.label["value"]
#             label = labels_dict[region]
#             label.text_alpha = 1 if (renderer.visible and button.active) else 0

#     def toggle_labels_callback(active):
#         update_labels_visibility()

#     button.on_click(toggle_labels_callback)

#     def legend_callback(attr, old, new):
#         update_labels_visibility()

#     for renderer in circle_renderers.values():
#         renderer.on_change("visible", legend_callback)

#     layout = column(button, p1)
#     return layout

# layout = create_layout()
# curdoc().add_root(layout)




### COLOURED BY URBAN-RURAL CLASSIFICATION

In [11]:
# from bokeh.transform import factor_cmap, factor_mark
# from bokeh.plotting import figure, show
# from bokeh.palettes import Category10



# def create_layout_p2(df2):
    
#     source = ColumnDataSource(df2)
#     tool = [
#         ("index", "$index"),
#         ("(x,y)", "($x, $y)"),
#         ("name","@LA_name"),
#     ]

#     urban = df2.Urb_Rur.unique()

#     p2 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
#                 x_axis_label="Non-response Rate",
#                 y_axis_label="Percentage of Non-English Speakers",
#                 tooltips=tool)

#     p2.scatter("Percentage", "Non-response_rate", source=df2, fill_alpha=0.5, size=10,
#                color=factor_cmap('Urb_Rur', Category10[10], urban), legend_field='Urb_Rur')

#     layout = column(p2)
#     return layout

# layout = create_layout_p2(df2)
# curdoc().add_root(layout)

### COLOURED BY SHANNON INDEX

Here, I have calculated the religious diversity index for each LA using the Shannon index.

In [12]:
# from bokeh.models import ColorBar, BasicTicker, PrintfTickFormatter
# from bokeh.models import LogColorMapper
# from bokeh.layouts import column
# from bokeh.models import ColumnDataSource

# def create_layout_p3(df):
#     color_map = LogColorMapper(palette="Viridis256", low=df.Shannon_idx.min(), high=df.Shannon_idx.max())

#     tool = [
#         ("index", "$index"),
#         ("(x,y)", "($x, $y)"),
#         ("name", "@LA_name"),
#         ("Shannon_idx", "@Shannon_idx")
#     ]

#     p3 = figure(title="Relationship between Non-response Rate and Non-English Speakers",
#                 x_axis_label="Non-response Rate",
#                 y_axis_label="Percentage of Non-English Speakers",
#                 tooltips=tool)

#     p3.scatter("Percentage", "Non-response_rate", source=df, fill_alpha=0.5, size=10,
#                color={'field': 'Shannon_idx', 'transform': color_map})

#     color_bar = ColorBar(color_mapper=color_map,
#                          title='Shannon Index',
#                          ticker=BasicTicker(desired_num_ticks=5),
#                          formatter=PrintfTickFormatter(format='%.2f'))

#     p3.add_layout(color_bar, 'right')

#     layout = column(p3)
#     return layout

# layout = create_layout_p3(df)
# curdoc().add_root(layout)


In [13]:
# Read-in pre-processed data for religion

rel = pd.read_csv('/Users/loucap/Documents/GitWork/InteractiveGender/Data/cleaned_religion_SO.csv')

In [14]:
rel

,LA_code,LA_name,SO_code,SO_categories,Religion_code,Religion_categories,Observation,Group_Percentages_No religion,Total_LA_counts,No religion_%,...,Observation_Hindu,Group_Percentages_Hindu,Total_LA_counts_Hindu,Hindu_%,Religion_code_Sikh,Religion_categories_Sikh,Observation_Sikh,Group_Percentages_Sikh,Total_LA_counts_Sikh,Sikh_%
0,E06000001,Hartlepool,5,Not answered,1,No religion,1139,1.61,70898,38.76,...,23,0.03,70898,0.23,7,Sikh,15,0.02,70898,0.19
1,E06000002,Middlesbrough,5,Not answered,1,No religion,1751,1.63,107747,35.98,...,86,0.08,107747,1.20,7,Sikh,33,0.03,107747,0.41
2,E06000003,Redcar and Cleveland,5,Not answered,1,No religion,1824,1.71,106453,38.58,...,9,0.01,106453,0.10,7,Sikh,4,0.00,106453,0.04
3,E06000004,Stockton-on-Tees,5,Not answered,1,No religion,2249,1.50,150094,38.48,...,26,0.02,150094,0.41,7,Sikh,25,0.02,150094,0.40
4,E06000005,Darlington,5,Not answered,1,No religion,1256,1.50,83761,39.32,...,21,0.03,83761,0.41,7,Sikh,30,0.04,83761,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,W06000020,Torfaen,5,Not answered,1,No religion,1495,2.12,70598,50.32,...,12,0.02,70598,0.32,7,Sikh,7,0.01,70598,0.06
327,W06000021,Monmouthshire,5,Not answered,1,No religion,1458,1.98,73676,43.34,...,14,0.02,73676,0.22,7,Sikh,6,0.01,73676,0.11
328,W06000022,Newport,5,Not answered,1,No religion,2386,1.98,120275,43.65,...,42,0.03,120275,0.48,7,Sikh,11,0.01,120275,0.27
329,W06000023,Powys,5,Not answered,1,No religion,2541,2.43,104686,42.47,...,25,0.02,104686,0.23,7,Sikh,3,0.00,104686,0.05


In [15]:
rel.columns

Index(['LA_code', 'LA_name', 'SO_code', 'SO_categories', 'Religion_code',
       'Religion_categories', 'Observation', 'Group_Percentages_No religion',
       'Total_LA_counts', 'No religion_%', 'Religion_code_Christian',
       'Religion_categories_Christian', 'Observation_Christian',
       'Group_Percentages_Christian', 'Total_LA_counts_Christian',
       'Christian_%', 'Religion_code_Muslim', 'Religion_categories_Muslim',
       'Observation_Muslim', 'Group_Percentages_Muslim',
       'Total_LA_counts_Muslim', 'Muslim_%', 'Religion_code_Other',
       'Religion_categories_Other', 'Observation_Other',
       'Group_Percentages_Other', 'Total_LA_counts_Other', 'Other_%',
       'Religion_code_Buddhist', 'Religion_categories_Buddhist',
       'Observation_Buddhist', 'Group_Percentages_Buddhist',
       'Total_LA_counts_Buddhist', 'Buddhist_%', 'Religion_code_Jewish',
       'Religion_categories_Jewish', 'Observation_Jewish',
       'Group_Percentages_Jewish', 'Total_LA_counts_Jewish',

In [16]:
# Read-in totals and non-response by religion

totals = pd.read_csv('/Users/loucap/Documents/GitWork/InteractiveGender/Data/gen_totals_SO.csv')

In [17]:
totals.head()

,Religion_categories,Observation,Percent_of_survey_respondents
0,Buddhist,245514,0.54
1,Christian,23656564,51.78
2,Hindu,823132,1.80
3,Jewish,213617,0.47
4,Muslim,2664709,5.83


In [18]:
totals = totals.sort_values(by = "Percent_of_survey_respondents", ascending = False)

In [19]:
# Read-in Non-response table

nr_totals = pd.read_csv('/Users/loucap/Documents/GitWork/InteractiveGender/Data/nr_totals_SO.csv')

nr_totals.head()

,Religion_categories,Observation,Non_response_rate,Contribution_to_overall_non_response_rate
0,Buddhist,18331,7.47,0.82
1,Christian,1054454,4.46,47.25
2,Hindu,61226,7.44,2.74
3,Jewish,18054,8.45,0.81
4,Muslim,217981,8.18,9.77


In [20]:
nr_totals = nr_totals.sort_values(by = "Contribution_to_overall_non_response_rate", ascending = False)

In [21]:
import pandas as pd
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, HTMLTemplateFormatter
from bokeh.models.widgets import DataTable, TableColumn, Div
from bokeh.plotting import figure, show, curdoc
from bokeh.io import output_notebook

# Custom cell formatter
template = """
<% if (Religion_categories == selected_religion) { %>
    <span style="color: red; font-weight: bold"><%= value %></span>
<% } else { %>
    <span style="font-weight: bold"><%= value %></span>
<% } %>
"""

def create_formatter(selected_religion):
    formatter = HTMLTemplateFormatter(template=template.replace("selected_religion", f"'{selected_religion}'"))
    return formatter

# Create DataTable for layout1
source1 = ColumnDataSource(totals)

columns1 = [
    TableColumn(field="Religion_categories", title="Religion", formatter=create_formatter('Christian')),
    TableColumn(field="Observation", title="Observation", formatter=create_formatter('Christian')),
    TableColumn(field="Percent_of_survey_respondents", title="% of respondents", formatter=create_formatter('Christian')),
]

heading1 = Div(text="<h1>Totals</h1>", width=300)

data_table1 = DataTable(source=source1, columns=columns1, editable=False, width=500, index_position=None)

layout1 = column(heading1, data_table1)

# Create DataTable for layout2
source2 = ColumnDataSource(nr_totals)

columns2 = [
    TableColumn(field="Religion_categories", title="Religion", formatter=create_formatter('Christian')),
    TableColumn(field="Observation", title="Observation", formatter=create_formatter('Christian')),
    TableColumn(field="Non_response_rate", title="Non response rate", formatter=create_formatter('Christian')),
    TableColumn(field="Contribution_to_overall_non_response_rate", title="% of total Non-response rate", formatter=create_formatter('Christian')),
]

heading2 = Div(text="<h1>Non-response rates</h1>", width=300)

data_table2 = DataTable(source=source2, columns=columns2, editable=False, width=700, index_position=None)

layout2 = column(heading2, data_table2)

# Scatter plot
output_notebook()

# Prepare data
rel['selected_religion'] = rel['Christian_%']  # Default religion
rel['selected_percentages'] = rel['Group_Percentages_Christian']

source = ColumnDataSource(rel)

# Define tooltips
tool = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("name", "@LA_name"),
]

# Create figure
p2 = figure(title="Relationship between % of religious group in given LA, and their non-response rate",
            y_axis_label="Non-response Rate", x_axis_label="Percentage of religious group in given LA", tooltips=tool)

# Scatter plot
p2.scatter("selected_religion", "selected_percentages", source=source, fill_alpha=0.5, size=10)

def update_highlighted_rows(selected_religion):
    formatter = create_formatter(selected_religion)
    for col in columns1:
        col.formatter = formatter
    for col in columns2:
        col.formatter = formatter
    data_table1.columns = columns1
    data_table2.columns = columns2

# Define callback for updating data source

def update_plot(attr, old, new):
    selected_religion = select_religion.value
    rel['selected_religion'] = rel[f'{selected_religion}_%']
    rel['selected_percentages'] = rel[f'Group_Percentages_{selected_religion}']
    source.data = source.from_df(rel)
    update_highlighted_rows(selected_religion)

# Create select widget
options = ['Christian', 'No religion', 'Muslim', 'Jewish', 'Buddhist', 'Hindu', 'Sikh', 'Other']
select_religion = Select(title="Religious Group:", value='Christian', options=options)
select_religion.on_change('value', update_plot)

# Initial update of the highlighted rows
update_highlighted_rows(select_religion.value)

# Layout
layout = column(select_religion, p2)
l = row(layout1, layout2)

# Show plot
curdoc().add_root(column(layout, l))

Loading BokehJS ...

In [22]:
rel

,LA_code,LA_name,SO_code,SO_categories,Religion_code,Religion_categories,Observation,Group_Percentages_No religion,Total_LA_counts,No religion_%,...,Total_LA_counts_Hindu,Hindu_%,Religion_code_Sikh,Religion_categories_Sikh,Observation_Sikh,Group_Percentages_Sikh,Total_LA_counts_Sikh,Sikh_%,selected_religion,selected_percentages
0,E06000001,Hartlepool,5,Not answered,1,No religion,1139,1.61,70898,38.76,...,70898,0.23,7,Sikh,15,0.02,70898,0.19,59.05,2.33
1,E06000002,Middlesbrough,5,Not answered,1,No religion,1751,1.63,107747,35.98,...,107747,1.20,7,Sikh,33,0.03,107747,0.41,52.44,2.37
2,E06000003,Redcar and Cleveland,5,Not answered,1,No religion,1824,1.71,106453,38.58,...,106453,0.10,7,Sikh,4,0.00,106453,0.04,59.93,2.48
3,E06000004,Stockton-on-Tees,5,Not answered,1,No religion,2249,1.50,150094,38.48,...,150094,0.41,7,Sikh,25,0.02,150094,0.40,57.05,2.04
4,E06000005,Darlington,5,Not answered,1,No religion,1256,1.50,83761,39.32,...,83761,0.41,7,Sikh,30,0.04,83761,0.40,57.56,2.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,W06000020,Torfaen,5,Not answered,1,No religion,1495,2.12,70598,50.32,...,70598,0.32,7,Sikh,7,0.01,70598,0.06,47.93,2.43
327,W06000021,Monmouthshire,5,Not answered,1,No religion,1458,1.98,73676,43.34,...,73676,0.22,7,Sikh,6,0.01,73676,0.11,54.74,2.52
328,W06000022,Newport,5,Not answered,1,No religion,2386,1.98,120275,43.65,...,120275,0.48,7,Sikh,11,0.01,120275,0.27,48.28,2.09
329,W06000023,Powys,5,Not answered,1,No religion,2541,2.43,104686,42.47,...,104686,0.23,7,Sikh,3,0.00,104686,0.05,55.51,3.29
